# Preprocessing data

### Standardization, or mean removal and variance scaling


Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn;

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.



In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd

X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

X_scaled = preprocessing.scale(X_train)   # default axis=0 standardize each feature

X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

The standard score of a sample x is calculated as:
<tt>z = (x - u) / s</tt>
where u is the mean of the training samples or zero if <tt>with_mean=False</tt>, and s is the standard deviation of the training samples or one if <tt>with_std=False</tt>.


The preprocessing module further provides a utility class StandardScaler that implements the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.

In [2]:
sss = preprocessing.StandardScaler()
scaler = sss.fit(X_train)

In [3]:
scaler.transform(X_train)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [4]:
scaler.transform(X_train).mean(), scaler.transform(X_train).std()

(np.float64(4.9343245538895844e-17), np.float64(1.0))

The scaler instance can then be used on new data to transform it the same way it did on the training set



In [ ]:
X_test = np.array([[ 2., -3.,  2.],
                    [ 2.,  5.,  0.],
                    [ 0.,  1., -1.]])

In [ ]:
scaler.transform(X_test)

In [ ]:
scaler.fit(X_train)
scaler.transform(X_train)

In [ ]:
scaler.fit_transform(X_train)

### Scaling features to a range

An alternative standardization is scaling features to lie between a given minimum and maximum value, often between zero and one, or so that the maximum absolute value of each feature is scaled to unit size.

In [ ]:
X_train = np.array([[ 1., -1.,  2.],
                    [ 2.,  0.,  0.],
                    [ 0.,  1., -1.]])

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler((0,1)) # default [0,1]

In [ ]:
min_max_scaler.fit(X_train)

In [ ]:
min_max_scaler.transform(X_train)

In [ ]:
X_test = np.array([[-3., -1.,  4.]])
min_max_scaler.transform(X_test)

The transformation is given by:

X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

X_scaled = X_std * (max - min) + min

If your data contains many outliers, scaling using the mean and variance of the data is likely to not work very well. In these cases, you can use robust_scale and RobustScaler (x_scaled = x / max(abs(x)))
as drop-in replacements instead. They use more robust estimates for the center and range of your data.



In [5]:
from sklearn.preprocessing import RobustScaler
X = [[ 1., -2.,  2.],
     [ -2.,  1.,  3.],
     [ 4.,  1., -2.]]

In [6]:
transformer = RobustScaler().fit(X)

In [7]:
transformer.scale_

array([3. , 1.5, 2.5])

In [8]:
transformer.transform(X)

array([[ 0. , -2. ,  0. ],
       [-1. ,  0. ,  0.4],
       [ 1. ,  0. , -1.6]])

The transformation is computed as:

<tt>X_scaled = (X - X.median(axis=0)) / X.IQR(axis=0)</tt>

where <tt>IQR = q3 - q1</tt>

The IQR is the range between the 1st quartile (q1) and the 3rd quartile (q3).

## Normalization


Normalization is the process of scaling individual samples to have unit norm. This process can be useful if you plan to use a quadratic form such as the dot-product or any other kernel to quantify the similarity of any pair of samples.

‘l1’
The l1 norm uses the sum of all the values and thus gives equal penalty to all parameters, enforcing sparsity.
x_normalized = x / sum(X)


‘l2’
The l2 norm uses the square root of the sum of all the squared values. This creates smoothness and rotational invariance. Some models, like PCA, assume rotational invariance, and so l2 will perform better.
x_normalized = x / sqrt(sum((i**2) for i in X))

In [ ]:
X = np.array([[1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])
X_normalized = preprocessing.normalize(X, norm='l2')

In [ ]:
# the same as previous X_normalized = preprocessing.Normalizer(norm='l2').fit_transform(X)

In [ ]:
X_normalized

In [ ]:
np.sqrt(0.40824829**2 + 0.40824829**2 + 0.81649658**2)

In [ ]:
np.sqrt(0**2 + 0.70710678**2 + (-0.70710678)**2)

You can refer to [https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html) for more details.

##  Encoding categorical features

### OneHotEncoder

In [ ]:
X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
X

In [ ]:
pd.DataFrame(X, columns=['gender', 'locations', 'hobby'])

In [ ]:
enc = preprocessing.OneHotEncoder()
enc.fit(X)

In [ ]:
enc.transform(X).toarray()

In [ ]:
enc.get_feature_names_out()

In [ ]:
enc.transform([['male', 'Italy', 'football'],
               ['female', 'France', 'painting']]).toarray()

In [ ]:
enc.categories_

In [ ]:
new_features = []
for i in enc.categories_:
  for j in i:
    new_features.append(j)


Xnew = enc.transform(X).toarray()
pd.DataFrame(Xnew, columns=new_features)

#### with fixed categories value

In [ ]:
genders = ['female', 'male']
locations = ['Spain', 'France', 'Italy']
hobbies = ['football', 'painting', 'tennis']
enc = preprocessing.OneHotEncoder(categories=[genders, locations, hobbies])

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]

enc.fit(X)

In [ ]:
enc.transform([['female', 'USA', 'football']]).toarray()

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
enc.fit(X)

In [ ]:
enc.transform([['female', 'USA', 'football']]).toarray()

In [ ]:
enc.get_feature_names_out()

In [ ]:
enc.categories_

In [ ]:
enc = preprocessing.OneHotEncoder(handle_unknown='infrequent_if_exist', min_frequency=2)

X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]
enc.fit(X)

In [ ]:
enc.categories_

In [ ]:
enc.get_feature_names_out()

In [ ]:
enc.transform([['female', 'USA', 'football']]).toarray()

In [ ]:
enc.inverse_transform([[1., 0., 0., 0., 1., 1., 0., 0.]])

### OrdinalEncoder

In [ ]:
enc = preprocessing.OrdinalEncoder()

In [ ]:
X = [['male', 'Spain', 'tennis'],
     ['female', 'Italy', 'football'],
     ['female', 'Italy', 'painting'],
     ['male', 'Spain', 'tennis'],
     ['female', 'France', 'football'],
     ['male', 'Italy', 'painting']]

enc.fit(X)

In [ ]:
enc.categories_

In [ ]:
enc.transform(X)

In [ ]:
enc.transform([['female','Spain', 'football']])

## Discretization

Discretization (otherwise known as quantization or binning) provides a way to partition **continuous features into discrete values**. Certain datasets with continuous features may benefit from discretization, because discretization can transform the dataset of continuous attributes to one with only nominal attributes.

In [ ]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 3], encode='ordinal').fit(X)
est

In [ ]:
est.transform(X)

There are different strategies implemented in KBinsDiscretizer:

- ‘uniform’: The discretization is uniform in each feature, which means that the bin widths are constant in each dimension.

- ‘quantile’: The discretization is done on the quantiled values, which means that each bin has approximately the same number of samples.

- ‘kmeans’: The discretization is based on the centroids of a KMeans clustering procedure.

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html?highlight=kbinsdiscretizer

Feature binarization is the process of thresholding numerical features to get boolean values. This can be useful for downstream probabilistic estimators that make assumption that the input data is distributed according to a multi-variate Bernoulli distribution.

In [ ]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

binarizer = preprocessing.Binarizer().fit(X)  # threshold default value = 0
binarizer

In [ ]:
binarizer.transform(X)

In [ ]:
binarizer = preprocessing.Binarizer(threshold=1.1)

In [ ]:
binarizer.transform(X)

## Custom transformers

Often, you will want to convert an existing Python function into a transformer to assist in data cleaning or processing.

In [ ]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p, validate=True)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)

The *validate* parameter indicates that the input X array should be checked before calling func. The possibilities are:

- If False, there is no input validation.

- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised.

In [ ]:
type(X)

In [ ]:
type(transformer.transform(X))

In [ ]:
df = pd.DataFrame(X)

In [ ]:
type(df)

In [ ]:
type(transformer.transform(df))

In [ ]:
transformer = FunctionTransformer(np.log1p, validate=False)

In [ ]:
type(transformer.transform(df))

**Pay Attention!
The result of a transformer is typically a np.array!**

In [ ]:
preprocessing.StandardScaler().fit_transform(df)

In [ ]:
preprocessing.StandardScaler().fit_transform(X)